# BentoML Example: Tensorflow 2.0 example (Echo model)

[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment. 

This notebook demonstrates how to use BentoML to turn a Tensorflow model into a docker image containing a REST API server serving this model, how to use your ML service built with BentoML as a CLI tool, and how to distribute it a pypi package.


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np
print(tf.__version__)

import os
import time
import requests
import json

2.0.0


In [4]:
class EchoModel(tf.keras.Model):
    def call(self, x):
        return tf.multiply(x, 1)

custom_model = EchoModel()
custom_model.compile(optimizer='sgd',
              loss="mean_squared_error",
              metrics=['accuracy'])

test_input =  tf.constant(np.zeros([1, 2, 2]))
test_output = tf.constant(np.zeros([1, 2, 2]))

custom_model.fit(test_input, test_output, epochs=1)  # required. it will generate the signature automaticlly

# test
custom_model(tf.constant(np.ones([4, 2, 3]), dtype=tf.float32))

Train on 1 samples
1/1 [==============================] - 0s 87ms/sample - loss: 0.0000e+00 - accuracy: 1.0000


<tf.Tensor: id=192, shape=(4, 2, 3), dtype=float32, numpy=
array([[[1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.]]], dtype=float32)>

In [6]:
test_tensor = tf.constant(np.zeros([2,4,1]), dtype=tf.float32)
custom_model(test_tensor)

# Serve with bentoml
----

In [11]:
%%writefile tensorflow_echo.py

import bentoml
import tensorflow as tf
import numpy as np
from PIL import Image

from bentoml.artifact import (
    TensorflowSavedModelArtifact,
)
from bentoml.handlers import TensorflowTensorHandler


FASHION_MNIST_CLASSES = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


@bentoml.env(pip_dependencies=['tensorflow', 'numpy', 'scikit-learn'])
@bentoml.artifacts([TensorflowSavedModelArtifact('model')])
class EchoServicer(bentoml.BentoService):

    @bentoml.api(TensorflowTensorHandler)
    def predict(self, tensor):
        outputs = self.artifacts.model(tensor)
        return outputs


Overwriting tensorflow_echo.py


In [12]:
# save model
from tensorflow_echo import EchoServicer
bento_svc = EchoServicer()
bento_svc.pack("model", custom_model)
saved_path = bento_svc.save()

[2019-12-20 11:36:37,121] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2019-12-20 11:36:37,122] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'examples'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'docs'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check


creating BentoML-0.5.2+41.g2222433.dirty
creating BentoML-0.5.2+41.g2222433.dirty/BentoML.egg-info
creating BentoML-0.5.2+41.g2222433.dirty/bentoml
creating BentoML-0.5.2+41.g2222433.dirty/bentoml/artifact
creating BentoML-0.5.2+41.g2222433.dirty/bentoml/bundler
creating BentoML-0.5.2+41.g2222433.dirty/bentoml/cli
creating BentoML-0.5.2+41.g2222433.dirty/bentoml/clipper
creating BentoML-0.5.2+41.g2222433.dirty/bentoml/configuration
creating BentoML-0.5.2+41.g2222433.dirty/bentoml/deployment
creating BentoML-0.5.2+41.g2222433.dirty/bentoml/deployment/aws_lambda
creating BentoML-0.5.2+41.g2222433.dirty/bentoml/deployment/sagemaker
creating BentoML-0.5.2+41.g2222433.dirty/bentoml/handlers
creating BentoML-0.5.2+41.g2222433.dirty/bentoml/migrations
creating BentoML-0.5.2+41.g2222433.dirty/bentoml/migrations/versions
creating BentoML-0.5.2+41.g2222433.dirty/bentoml/proto
creating BentoML-0.5.2+41.g2222433.dirty/bentoml/repository
creating BentoML-0.5.2+41.g2222433.dirty/bentoml/server
creat

### Test packed BentoML service

In [33]:
bento_svc.predict([1, 2, 3])

<tf.Tensor: id=599, shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

### Run REST API server locally

In [21]:
!bentoml serve {saved_path}

[2019-12-20 11:40:23,757] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2019-12-20 11:40:23,768] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.2,  but loading from BentoML version 0.5.2+41.g2222433.dirty
[2019-12-20 11:40:24,852] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image 

### Send prediction request to REST API server

*Run the following command in terminal to make a HTTP request to the API server*
```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '{"instances": [[1, 2]]}' \
localhost:5000/predict
```

In [22]:
import requests
import json
headers = {"content-type": "application/json"}
data = json.dumps(
    {"instances": [[1, 2, 2, 3], [2, 3, 3, 4]]}
)
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))
json_response = requests.post(f'http://127.0.0.1:5000/predict', data=data, headers=headers)
print(json_response)
print(json_response.text)


Data: {"instances": [[1, 2, 2, 3], [2, 3, 3, 4]]} ... , 3, 4]]}
<Response [200]>
{"predictions": [[1, 2, 2, 3], [2, 3, 3, 4]]}


# "pip install" a BentoML archive

BentoML user can directly pip install saved BentoML archive with `pip install $SAVED_PATH`,  and use it as a regular python package.

In [ ]:
!pip install {saved_path}

In [19]:
import EchoServicer

installed_svc = EchoServicer.load()

In [20]:
installed_svc.predict({ 'instances': [[1, 2] })

array([[1]], dtype=int32)

# CLI access

`pip install $SAVED_PATH` also installs a CLI tool for accessing the BentoML service

In [ ]:
!EchoServicer --help

### Print model service information:

In [ ]:
!EchoServicer info

### Run 'predict' api with json data:

In [ ]:
!EchoServicer predict --input='{"instances": [[1, 2]]}'

Additional: Serve with tf-serving
----
Bentoml TensorFlow handler and artifact is following the API of tensorflow-serving REST API.  
To install tensorflow-serving, see: https://www.tensorflow.org/tfx/serving/setup


In [28]:
TMP_MODEL_DIR = "/tmp/test-echo-model"
TMP_MODEL_VERSION = "1"
TMP_MODEL_DIR_V = f"{TMP_MODEL_DIR}/{TMP_MODEL_VERSION}"
MODEL_NAME = "echo_model"

tf.saved_model.save(custom_model, TMP_MODEL_DIR_V)
!tensorflow_model_server --rest_api_port=5001 --model_name={MODEL_NAME} --model_base_path={TMP_MODEL_DIR}


INFO:tensorflow:Assets written to: /tmp/test-echo-model/2/assets
2019-12-20 12:03:01.458521: I tensorflow_serving/model_servers/server.cc:85] Building single TensorFlow model file config:  model_name: echo_model model_base_path: /tmp/test-echo-model
2019-12-20 12:03:01.458658: I tensorflow_serving/model_servers/server_core.cc:462] Adding/updating models.
2019-12-20 12:03:01.458673: I tensorflow_serving/model_servers/server_core.cc:573]  (Re-)adding model: echo_model
2019-12-20 12:03:01.559267: I tensorflow_serving/core/basic_manager.cc:739] Successfully reserved resources to load servable {name: echo_model version: 2}
2019-12-20 12:03:01.559323: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: echo_model version: 2}
2019-12-20 12:03:01.559349: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: echo_model version: 2}
2019-12-20 12:03:01.559384: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:31] Reading

In [32]:
import requests
import json

TMP_MODEL_DIR = "/tmp/test-echo-model"
TMP_MODEL_VERSION = "1"
TMP_MODEL_DIR_V = f"{TMP_MODEL_DIR}/{TMP_MODEL_VERSION}"
MODEL_NAME = "echo_model"
headers = {"content-type": "application/json"}
data = json.dumps(
    {"instances": [[1, 2, 2, 3], [2, 3, 3, 4]]}
)
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))
json_response = requests.post(f'http://127.0.0.1:5001/v{TMP_MODEL_VERSION}/models/{MODEL_NAME}:predict',
                              data=data, headers=headers)
print(json_response)
print(json_response.text)


Data: {"instances": [[1, 2, 2, 3], [2, 3, 3, 4]]} ... , 3, 4]]}
<Response [200]>
{
    "predictions": [[1.0, 2.0, 2.0, 3.0], [2.0, 3.0, 3.0, 4.0]
    ]
}
